In [ ]:
from util_debugger import get_sys_metric
from util_train import aug_exp_train
import pprint

### Setting up required parameters for the experiment to compare training time for augmentation on CPU and GPU

In [ ]:
# Options: ['pytorch-cpu', 'dali-gpu', 'dali-cpu'] 
AUGMENTATION_APPROACHES = ['pytorch-cpu', 'dali-gpu']

instance_type = 'ml.p3.2xlarge' #[1-GPU][8-CPU]
#instance_type = 'ml.g4dn.16xlarge' #[1-GPU][64-CPU]
#instance_type = 'ml.p3.8xlarge' #[4-GPU][32-CPU]
#instance_type = 'ml.g4dn.12xlarge' #[4-GPU] [48-CPU]

model_arch = 'RESNET18'
#model_arch = 'RESNET50'
#model_arch = 'RESNET152'

batch_size = 32

aug_load_factor = 12

### Launching training jobs and fetching system utilization

In [ ]:
exp_data = {}
trial = 0
pp = pprint.PrettyPrinter()

for aug_operator in AUGMENTATION_APPROACHES:
        
    trial = trial + 1
    trial_data = dict.fromkeys(['train_job_id', 'model_arch', 'instance_type', 'batch_size', 'aug_load_factor', 'aug_operator', 'sys_util_df'])
            
    train_job_id, train_estimator = aug_exp_train(model_arch, batch_size, aug_operator, aug_load_factor, instance_type)
    heatmap, metric_hist, sys_util_df = get_sys_metric(train_estimator, instance_type)
            
    trial_data['train_job_id'] = train_job_id
    trial_data['model_arch'] = model_arch
    trial_data['instance_type'] = instance_type
    trial_data['batch_size'] = batch_size
    trial_data['aug_load_factor'] = aug_load_factor
    trial_data['aug_operator'] = aug_operator
    trial_data['sys_util_df'] = sys_util_df
            
    pp.pprint(trial_data) 
    exp_data.update({'trial-'+str(trial): trial_data})
            
pp.pprint(exp_data) 